# A deep dive into DataFrames.jl indexing
# Part 1: indexing in DataFrames.jl by example
### Bogumił Kamiński

`DataFrame` is an object that holds a collection of named columns stored as vectors.

In this tutorial we discuss how to get or set values of these columns.

What are we going to cover:
* `getindex`, a.k.a. `x[...]`
* `setindex!`, a.k.a. `x[...] =`
* `broadcast`, a.k.a. `fun.(x)`
* `broadcast!`, a.k.a. `x .= ...`
* `getproperty`, a.k.a. `x.field` and `x.field .= ...`
* `setproperty`, a.k.a. `x.field = ...`

Indexable types that DataFrames.jl defines:
* `DataFrame`
* `SubDataFrame`
* `DataFrameRow`
* `DataFrameRows`
* `DataFrameColumns`
* `GroupedDataFrame`
* `GroupKeys`
* `GroupKey`
* `StackedVector`
* `RepeatedVector`

### Environment setup

In [1]:
using BenchmarkTools

In [2]:
using CSV

In [3]:
using DataFrames

In [4]:
using Dates

In [5]:
using ShiftedArrays

In [6]:
using Statistics

Typical initial setup steps to override default settings of JupyterNotebook

In [7]:
ENV["COLUMNS"] = 500

500

In [8]:
ENV["LINES"] = 15

15

Let us read-in the data set we will work with.

Make sure you have the required file in the working directory.

The detailed instructions how to get it are in https://github.com/bkamins/JuliaCon2020-DataFrames-Tutorial/blob/master/README.md

In [9]:
fh5 = CSV.read("fh_5yrs.csv", DataFrame)

,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String
1,2020-07-02,257500,17.64,17.74,17.62,17.71,17.71,AAAU
2,2020-07-01,468100,17.73,17.73,17.54,17.68,17.68,AAAU
3,2020-06-30,319100,17.65,17.8,17.61,17.78,17.78,AAAU
4,2020-06-29,405500,17.67,17.69,17.63,17.68,17.68,AAAU
5,2020-06-26,335100,17.49,17.67,17.42,17.67,17.67,AAAU
6,2020-06-25,246800,17.6,17.6,17.52,17.59,17.59,AAAU
7,2020-06-24,329200,17.61,17.71,17.56,17.61,17.61,AAAU
8,2020-06-23,351800,17.55,17.66,17.55,17.66,17.66,AAAU
9,2020-06-22,308300,17.5,17.57,17.44,17.5,17.5,AAAU


#### Warm up exercises

*Get a short description of columns in our data frame*

In [10]:
describe(fh5)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,date,,2015-01-02,,2020-07-02,0,Date
2,volume,1.01541e6,1,120600.0,2156725200,0,Int64
3,open,298.086,0.001,24.95,6.91553e7,0,Float64
4,high,305.876,0.0,25.11,7.05886e7,0,Float64
5,low,291.014,0.0,24.75,6.84387e7,0,Float64
6,close,296.783,0.001,24.94,6.95136e7,0,Float64
7,adjclose,293.231,-3.77096,23.3258,6.90222e7,0,Float64
8,symbol,,AAAU,,ZYXI,0,String


(see https://github.com/JuliaData/DataFrames.jl/issues/2269 for a discussion of the design decisions here, feel free to comment there if you have an opinion)

*Get information about exact types of the columns stored in the data frame*

In [11]:
typeof.(eachcol(fh5))

8-element Vector{DataType}:
 Vector{Date} (alias for Array{Date, 1})
 Vector{Int64} (alias for Array{Int64, 1})
 Vector{Float64} (alias for Array{Float64, 1})
 Vector{Float64} (alias for Array{Float64, 1})
 Vector{Float64} (alias for Array{Float64, 1})
 Vector{Float64} (alias for Array{Float64, 1})
 Vector{Float64} (alias for Array{Float64, 1})
 PooledArrays.PooledVector{String, UInt32, Vector{UInt32}} (alias for PooledArrays.PooledArray{String, UInt32, 1, Array{UInt32, 1}})

*Get names of columns as strings*

In [12]:
names(fh5)

8-element Vector{String}:
 "date"
 "volume"
 "open"
 "high"
 "low"
 "close"
 "adjclose"
 "symbol"

*Get names of columns as `Symbol`s*

In [13]:
propertynames(fh5)

8-element Vector{Symbol}:
 :date
 :volume
 :open
 :high
 :low
 :close
 :adjclose
 :symbol

## `getindex`

Get a single column as a whole without copying

In [14]:
unique([fh5.date,
        fh5."date",
        fh5[!, 1],
        fh5[!, :date],
        fh5[!, "date"]])

1-element Vector{Vector{Date}}:
 [Date("2020-07-02"), Date("2020-07-01"), Date("2020-06-30"), Date("2020-06-29"), Date("2020-06-26"), Date("2020-06-25"), Date("2020-06-24"), Date("2020-06-23"), Date("2020-06-22"), Date("2020-06-19")  …  Date("2015-01-21"), Date("2015-01-20"), Date("2015-01-16"), Date("2015-01-14"), Date("2015-01-13"), Date("2015-01-12"), Date("2015-01-09"), Date("2015-01-07"), Date("2015-01-05"), Date("2015-01-02")]

In [15]:
unique([getproperty(fh5, :date),
        getproperty(fh5, "date"),
        getindex(fh5, !, 1),
        getindex(fh5, !, :date),
        getindex(fh5,!, "date")])

1-element Vector{Vector{Date}}:
 [Date("2020-07-02"), Date("2020-07-01"), Date("2020-06-30"), Date("2020-06-29"), Date("2020-06-26"), Date("2020-06-25"), Date("2020-06-24"), Date("2020-06-23"), Date("2020-06-22"), Date("2020-06-19")  …  Date("2015-01-21"), Date("2015-01-20"), Date("2015-01-16"), Date("2015-01-14"), Date("2015-01-13"), Date("2015-01-12"), Date("2015-01-09"), Date("2015-01-07"), Date("2015-01-05"), Date("2015-01-02")]

Get a single column as a whole with copying

In [16]:
unique([copy(fh5.date),
        copy(fh5."date"),
        fh5[:, 1],
        fh5[:, :date],
        fh5[:, "date"]])

1-element Vector{Vector{Date}}:
 [Date("2020-07-02"), Date("2020-07-01"), Date("2020-06-30"), Date("2020-06-29"), Date("2020-06-26"), Date("2020-06-25"), Date("2020-06-24"), Date("2020-06-23"), Date("2020-06-22"), Date("2020-06-19")  …  Date("2015-01-21"), Date("2015-01-20"), Date("2015-01-16"), Date("2015-01-14"), Date("2015-01-13"), Date("2015-01-12"), Date("2015-01-09"), Date("2015-01-07"), Date("2015-01-05"), Date("2015-01-02")]

Let us compare the performance of various ways to get a column without copying

In [17]:
@btime $fh5.date
@btime $fh5."date"
@btime $fh5[!, 1]
@btime $fh5[!, :date]
@btime $fh5[!, "date"];

  17.048 ns (0 allocations: 0 bytes)
  72.929 ns (0 allocations: 0 bytes)
  4.635 ns (0 allocations: 0 bytes)
  15.473 ns (0 allocations: 0 bytes)
  59.548 ns (0 allocations: 0 bytes)


`@btime` is from BenchmarkTools.jl package. We use `$` to ensure the time is measured properly.
This is a special syntax specific to `@btime` (like `$` used in string interpolation context).

#### Exercise

Check the same but with copying

In [18]:
@btime copy($fh5.date)
@btime copy($fh5."date")
@btime $fh5[:, 1]
@btime $fh5[:, :date]
@btime $fh5[:, "date"];

  30.092 ms (2 allocations: 52.28 MiB)
  29.528 ms (2 allocations: 52.28 MiB)
  29.389 ms (2 allocations: 52.28 MiB)
  30.189 ms (2 allocations: 52.28 MiB)
  31.128 ms (2 allocations: 52.28 MiB)


Let us check how lookup speed scales with the number of columns:

In [19]:
@time df_tmp = DataFrame(ones(1, 100_000), :auto)

  0.119968 seconds (435.78 k allocations: 33.022 MiB, 20.92% compilation time)


,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [20]:
@btime $df_tmp.x100000
@btime $df_tmp."x100000"
@btime $df_tmp[!, 100000];

  17.429 ns (0 allocations: 0 bytes)
  87.443 ns (0 allocations: 0 bytes)
  5.027 ns (0 allocations: 0 bytes)


<div class="alert alert-block alert-info">
<b>Tip:</b>
    
DataFrames.jl is specifically designed to allow you to handle without huge compilation costs
very wide data frames with heterogeneous column types and changing the schema of the data frame in-place.
(or if you do not want to think if you will not run into these issues)
</div>

Get a single column, but take a subset of rows: you can either make a copy or get a view

In [21]:
fh5[1:2, :date]

2-element Vector{Date}:
 2020-07-02
 2020-07-01

In [22]:
view(fh5, 1:2, :date)

2-element view(::Vector{Date}, 1:2) with eltype Date:
 2020-07-02
 2020-07-01

this is the same as e.g.:

In [23]:
fh5.date[1:2]

2-element Vector{Date}:
 2020-07-02
 2020-07-01

In [24]:
@view fh5.date[1:2]

2-element view(::Vector{Date}, 1:2) with eltype Date:
 2020-07-02
 2020-07-01

you can use `Not` for inverted selection of rows

In [25]:
fh5[Not(3:end), :date]

2-element Vector{Date}:
 2020-07-02
 2020-07-01

Get a single cell in a data frame: you can either get a value or a view

In [26]:
fh5[1, :date]

2020-07-02

In [27]:
fh5[CartesianIndex(1, 1)]

2020-07-02

In [28]:
@view fh5[1, "date"]

0-dimensional view(::Vector{Date}, 1) with eltype Date:
Date("2020-07-02")

In what case you might want to use a view instead of getting a value?

Check what is the consequence of running the following lines:

In [29]:
tmp_cell = view(fh5, 1, :date)

0-dimensional view(::Vector{Date}, 1) with eltype Date:
Date("2020-07-02")

In [30]:
tmp_cell2 = getindex(fh5, 1, :date)

2020-07-02

In [31]:
tmp_cell[] = Date("2222-07-02")

2222-07-02

In [32]:
fh5

,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String
1,2222-07-02,257500,17.64,17.74,17.62,17.71,17.71,AAAU
2,2020-07-01,468100,17.73,17.73,17.54,17.68,17.68,AAAU
3,2020-06-30,319100,17.65,17.8,17.61,17.78,17.78,AAAU
4,2020-06-29,405500,17.67,17.69,17.63,17.68,17.68,AAAU
5,2020-06-26,335100,17.49,17.67,17.42,17.67,17.67,AAAU
6,2020-06-25,246800,17.6,17.6,17.52,17.59,17.59,AAAU
7,2020-06-24,329200,17.61,17.71,17.56,17.61,17.61,AAAU
8,2020-06-23,351800,17.55,17.66,17.55,17.66,17.66,AAAU
9,2020-06-22,308300,17.5,17.57,17.44,17.5,17.5,AAAU


Revert the change we have just made

In [33]:
tmp_cell[] = tmp_cell2

2020-07-02

In [34]:
fh5

,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String
1,2020-07-02,257500,17.64,17.74,17.62,17.71,17.71,AAAU
2,2020-07-01,468100,17.73,17.73,17.54,17.68,17.68,AAAU
3,2020-06-30,319100,17.65,17.8,17.61,17.78,17.78,AAAU
4,2020-06-29,405500,17.67,17.69,17.63,17.68,17.68,AAAU
5,2020-06-26,335100,17.49,17.67,17.42,17.67,17.67,AAAU
6,2020-06-25,246800,17.6,17.6,17.52,17.59,17.59,AAAU
7,2020-06-24,329200,17.61,17.71,17.56,17.61,17.61,AAAU
8,2020-06-23,351800,17.55,17.66,17.55,17.66,17.66,AAAU
9,2020-06-22,308300,17.5,17.57,17.44,17.5,17.5,AAAU


To conclude note that with `view` there is no difference between `!` and `:`:

In [35]:
@view fh5[!, 1]

6852038-element view(::Vector{Date}, :) with eltype Date:
 2020-07-02
 2020-07-01
 2020-06-30
 2020-06-29
 2020-06-26
 ⋮
 2015-01-12
 2015-01-09
 2015-01-07
 2015-01-05
 2015-01-02

In [36]:
@view fh5[:, 1]

6852038-element view(::Vector{Date}, :) with eltype Date:
 2020-07-02
 2020-07-01
 2020-06-30
 2020-06-29
 2020-06-26
 ⋮
 2015-01-12
 2015-01-09
 2015-01-07
 2015-01-05
 2015-01-02

#### Exercise

Why is it useful to support `@view` both for `!` and `:`?

*Answer: because it makes it easy to use @views on whole expressions*

<div class="alert alert-block alert-info">
<b>Tip:</b>
    
passing a single column as an integer, `Symbol` or string drops one dimension of
a data frame and allows you to select or subset a column from it (depending on the row selector you choose)
</div>

Multiple column selection options are:
* a vector of `Symbol` (does not have to be a subtype of `AbstractVector{Symbol}`, e.g. `Any[:date]`);
* a vector of `AbstractString` (does not have to be a subtype of `AbstractVector{<:AbstractString}`, e.g. `Any["date"]`);
* a vector of `Integer` other than `Bool` (does not have to be a subtype of `AbstractVector{<:Integer}`, e.g. `Any[1]`);
* a vector of `Bool` that has to be a subtype of `AbstractVector{Bool}`;
* a regular expression, which gets expanded to a vector of matching column names;
* a `Not` expression;
* an `All` or `Between` expression;
* a `Cols` expression;
* a colon literal `:`.

The type of the result depends on the row selecor:
* if it is a single row you get a `DataFrameRow` (a dimension is dropped)
* if it is a collection of rows you get a data frame

Single row selection is always a view that is `DataFrameRow`:

In [37]:
fh5[1, [:date]]

,date
,Date
1,2020-07-02


In [38]:
@view fh5[1, [:date]]

,date
,Date
1,2020-07-02


Note that `DataFrameRow` is one-dimensional (as usual - single value indexing drops a dimension). You can think of it as a mutable `NamedTuple`.

Making `fg5[1, [:date]]` to be copying would incur overhead that was considered to be not justified by typical use cases of this syntax. (but if you disagree please open an issue on GitHub)

Multiple row selection is a `DataFrame` for `getindex`:

In [39]:
fh5[1:2, 1:2]

,date,volume
,Date,Int64
1,2020-07-02,257500
2,2020-07-01,468100


This does not copy columns and is fast

In [40]:
df_tmp = fh5[!, 1:2]

,date,volume
,Date,Int64
1,2020-07-02,257500
2,2020-07-01,468100
3,2020-06-30,319100
4,2020-06-29,405500
5,2020-06-26,335100
6,2020-06-25,246800
7,2020-06-24,329200
8,2020-06-23,351800
9,2020-06-22,308300


In [41]:
df_tmp.date === fh5.date

true

Using `view` creates a `SubDataFrame`

In [42]:
dfv_tmp = view(fh5, 1:2, 1:2)

,date,volume
,Date,Int64
1,2020-07-02,257500
2,2020-07-01,468100


In [43]:
typeof(dfv_tmp)

SubDataFrame{DataFrame, DataFrames.SubIndex{DataFrames.Index, UnitRange{Int64}, UnitRange{Int64}}, UnitRange{Int64}}

#### Execrcise

Check that `view(fh5, !, :)` and `view(fh5, :, :)` produce the same result

In [44]:
dump(view(fh5, !, :), maxdepth=2)

SubDataFrame{DataFrame, DataFrames.Index, Base.OneTo{Int64}}
  parent: DataFrame
    columns: Array{AbstractVector{var"#s831"} where var"#s831"}((8,))
    colindex: DataFrames.Index
  colindex: DataFrames.Index
    lookup: Dict{Symbol, Int64}
    names: Array{Symbol}((8,))
  rows: Base.OneTo{Int64}
    stop: Int64 6852038


In [45]:
dump(view(fh5, :, :), maxdepth=2)

SubDataFrame{DataFrame, DataFrames.Index, Base.OneTo{Int64}}
  parent: DataFrame
    columns: Array{AbstractVector{var"#s831"} where var"#s831"}((8,))
    colindex: DataFrames.Index
  colindex: DataFrames.Index
    lookup: Dict{Symbol, Int64}
    names: Array{Symbol}((8,))
  rows: Base.OneTo{Int64}
    stop: Int64 6852038


As a warning remember that when you modify the parent of a `SubDataFrame` (or `DataFrameRow`) you may get an error when trying to access it:

In [46]:
df_tmp = fh5[1:3, 1:4]

,date,volume,open,high
,Date,Int64,Float64,Float64
1,2020-07-02,257500,17.64,17.74
2,2020-07-01,468100,17.73,17.73
3,2020-06-30,319100,17.65,17.8


In [47]:
dfv_tmp = view(df_tmp, 1:2, 1:3)

,date,volume,open
,Date,Int64,Float64
1,2020-07-02,257500,17.64
2,2020-07-01,468100,17.73


In [48]:
select!(df_tmp, 1) # note that in `select` et al. a data frame is always produced

,date
,Date
1,2020-07-02
2,2020-07-01
3,2020-06-30


In [49]:
dfv_tmp

BoundsError: BoundsError: attempt to access 1-element Vector{Symbol} at index [1:3]

A special case is when you use `:` as a column selection with a `view`. In this case the `SubDataFrame` and `DataFrameRow` always get updated with the changed columns:

In [50]:
df_tmp = fh5[1:3, 1:4]

,date,volume,open,high
,Date,Int64,Float64,Float64
1,2020-07-02,257500,17.64,17.74
2,2020-07-01,468100,17.73,17.73
3,2020-06-30,319100,17.65,17.8


In [51]:
dfv_tmp = view(df_tmp, 1:2, :)

,date,volume,open,high
,Date,Int64,Float64,Float64
1,2020-07-02,257500,17.64,17.74
2,2020-07-01,468100,17.73,17.73


In [52]:
select!(df_tmp, 1, :open => :newcol)

,date,newcol
,Date,Float64
1,2020-07-02,17.64
2,2020-07-01,17.73
3,2020-06-30,17.65


In [53]:
dfv_tmp

,date,newcol
,Date,Float64
1,2020-07-02,17.64
2,2020-07-01,17.73


The reason for this behavior is that subsetting of a data frame by only rows (and taking all columns) is very common, and in this case we can create and index such views much faster. In particular `DataFrameRow`s produced by `eachrow` are efficient this way:

In [54]:
@btime mean(x -> x.open, eachrow(fh5))

  1.039 s (34259694 allocations: 522.76 MiB)


298.08612911254846

In [55]:
@btime mean(i -> fh5[i, :open], 1:nrow(fh5))

  1.453 s (41111214 allocations: 627.31 MiB)


298.08612911254846

Of course, type-stable operation would be faster (but sometimes processing data row-wise is more convenient):

In [56]:
@btime mean(fh5.open)

  3.090 ms (1 allocation: 16 bytes)


298.08612911254846

or, if your table is not very wide (so that you are not penalized by the compilation cost of `NamedTuple`) you can use:

In [57]:
@btime mean(x -> x.open, Tables.namedtupleiterator(fh5))

  22.137 ms (13 allocations: 816 bytes)


298.0861291125547

<div class="alert alert-block alert-info">
<b>Tip:</b>
    

If your table is not very wide then you can use `Tables.namedtupleiterator` or `Tables.columntable` to swithch a `DataFrame` into a type-stable mode (at the cost of fixing its schema). Both are non-allocating. You can also used `Tables.rowtable` but it is allocating so usually `Tables.namedtupleiterator` is preferred.
</div>

Note though that `DataFrameRow` allows you to modify the source data frame, while iterating `NamedTuple`s is read-only (more on `setindex!` later).

In [58]:
df_tmp = copy(fh5)

,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String
1,2020-07-02,257500,17.64,17.74,17.62,17.71,17.71,AAAU
2,2020-07-01,468100,17.73,17.73,17.54,17.68,17.68,AAAU
3,2020-06-30,319100,17.65,17.8,17.61,17.78,17.78,AAAU
4,2020-06-29,405500,17.67,17.69,17.63,17.68,17.68,AAAU
5,2020-06-26,335100,17.49,17.67,17.42,17.67,17.67,AAAU
6,2020-06-25,246800,17.6,17.6,17.52,17.59,17.59,AAAU
7,2020-06-24,329200,17.61,17.71,17.56,17.61,17.61,AAAU
8,2020-06-23,351800,17.55,17.66,17.55,17.66,17.66,AAAU
9,2020-06-22,308300,17.5,17.57,17.44,17.5,17.5,AAAU


#### Exercise

In `df_tmp` find rows in which `:high` is less than `:low` and swap these values.
We will discuss three ways to do it.

In [59]:
bad_idx = findall(df_tmp.low .> df_tmp.high)
@show df_tmp[bad_idx, :]
df_tmp.low[bad_idx], df_tmp.high[bad_idx] = df_tmp.high[bad_idx], df_tmp.low[bad_idx]
@show df_tmp[bad_idx, :]

df_tmp[bad_idx, :] = 2×8 DataFrame
 Row │ date        volume  open     high     low      close    adjclose  symbol
     │ Date        Int64   Float64  Float64  Float64  Float64  Float64   String
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ 2016-10-19     468   6345.0   6250.0   6345.0   6250.0    6250.0  PALC
   2 │ 2016-10-07   88860   6250.0   6150.0   6250.0   6150.0    6150.0  PALC
df_tmp[bad_idx, :] = 2×8 DataFrame
 Row │ date        volume  open     high     low      close    adjclose  symbol
     │ Date        Int64   Float64  Float64  Float64  Float64  Float64   String
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ 2016-10-19     468   6345.0   6345.0   6250.0   6250.0    6250.0  PALC
   2 │ 2016-10-07   88860   6250.0   6250.0   6150.0   6150.0    6150.0  PALC


,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String
1,2016-10-19,468,6345.0,6345.0,6250.0,6250.0,6250.0,PALC
2,2016-10-07,88860,6250.0,6250.0,6150.0,6150.0,6150.0,PALC


In [60]:
df_tmp = copy(fh5)
@show df_tmp[bad_idx, :]
foreach(eachrow(df_tmp)) do row
    if row.low >  row.high
        row.low, row.high = row.high, row.low
    end
end
@show df_tmp[bad_idx, :]

df_tmp[bad_idx, :] = 2×8 DataFrame
 Row │ date        volume  open     high     low      close    adjclose  symbol
     │ Date        Int64   Float64  Float64  Float64  Float64  Float64   String
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ 2016-10-19     468   6345.0   6250.0   6345.0   6250.0    6250.0  PALC
   2 │ 2016-10-07   88860   6250.0   6150.0   6250.0   6150.0    6150.0  PALC
df_tmp[bad_idx, :] = 2×8 DataFrame
 Row │ date        volume  open     high     low      close    adjclose  symbol
     │ Date        Int64   Float64  Float64  Float64  Float64  Float64   String
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ 2016-10-19     468   6345.0   6345.0   6250.0   6250.0    6250.0  PALC
   2 │ 2016-10-07   88860   6250.0   6250.0   6150.0   6150.0    6150.0  PALC


,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String
1,2016-10-19,468,6345.0,6345.0,6250.0,6250.0,6250.0,PALC
2,2016-10-07,88860,6250.0,6250.0,6150.0,6150.0,6150.0,PALC


In [61]:
df_tmp = copy(fh5)
@show df_tmp[bad_idx, :]
transform!(df_tmp, [[:low, :high]] .=> ByRow.([min, max]) .=> [:low, :high])
@show df_tmp[bad_idx, :]

df_tmp[bad_idx, :] = 2×8 DataFrame
 Row │ date        volume  open     high     low      close    adjclose  symbol
     │ Date        Int64   Float64  Float64  Float64  Float64  Float64   String
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ 2016-10-19     468   6345.0   6250.0   6345.0   6250.0    6250.0  PALC
   2 │ 2016-10-07   88860   6250.0   6150.0   6250.0   6150.0    6150.0  PALC
df_tmp[bad_idx, :] = 2×8 DataFrame
 Row │ date        volume  open     high     low      close    adjclose  symbol
     │ Date        Int64   Float64  Float64  Float64  Float64  Float64   String
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ 2016-10-19     468   6345.0   6345.0   6250.0   6250.0    6250.0  PALC
   2 │ 2016-10-07   88860   6250.0   6250.0   6150.0   6150.0    6150.0  PALC


,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String
1,2016-10-19,468,6345.0,6345.0,6250.0,6250.0,6250.0,PALC
2,2016-10-07,88860,6250.0,6250.0,6150.0,6150.0,6150.0,PALC


I give you the following column selectors. Can you tell the effect of each of them when trying to run `fh5[1:2, selector]`?
Write the code that tests it.

In [62]:
selectors = [Between(1, 10), Between(:low, :high), [:low, :low], Cols(:low, :low), Cols(:low, :), All()]

6-element Vector{Any}:
 Between{Int64, Int64}(1, 10)
 Between{Symbol, Symbol}(:low, :high)
 [:low, :low]
 Cols{Tuple{Symbol, Symbol}}((:low, :low))
 Cols{Tuple{Symbol, Colon}}((:low, Colon()))
 All{Tuple{}}(())

In [63]:
for selector in selectors
    @show selector
    try
        println(fh5[1:2, selector])
    catch
        println("errored")
    end
end

selector = Between{Int64, Int64}(1, 10)
errored
selector = Between{Symbol, Symbol}(:low, :high)
0×0 DataFrame
selector = [:low, :low]
errored
selector = Cols{Tuple{Symbol, Symbol}}((:low, :low))
2×1 DataFrame
 Row │ low     
     │ Float64 
─────┼─────────
   1 │   17.62
   2 │   17.54
selector = Cols{Tuple{Symbol, Colon}}((:low, Colon()))
2×8 DataFrame
 Row │ low      date        volume  open     high     close    adjclose  symbol 
     │ Float64  Date        Int64   Float64  Float64  Float64  Float64   String 
─────┼──────────────────────────────────────────────────────────────────────────
   1 │   17.62  2020-07-02  257500    17.64    17.74    17.71     17.71  AAAU
   2 │   17.54  2020-07-01  468100    17.73    17.73    17.68     17.68  AAAU
selector = All{Tuple{}}(())
2×8 DataFrame
 Row │ date        volume  open     high     low      close    adjclose  symbol 
     │ Date        Int64   Float64  Float64  Float64  Float64  Float64   String 
─────┼───────────────────────────────────

<div class="alert alert-block alert-info">
<b>Tip:</b>
    
In general `df.single_col` and `df[!, single_col]` produce the same result in `getindex`.

The exceptions are:
* `@view` and `@views` does not affect `df.single_col`.
* in `df.single_col` you cannot pass `single_col` as a variable (unless you write `getproperty(df, single_col)`)
* only `df[!, single_col]` allows integer indexing

</div>


### Indexing `GroupedDataFrame`

A `GroupedDataFrame` is a view into a data frame which defines a key allowing a fast lookup (and in particular this key is then automatically used in split-apply-combine operations with `select`, `select!`, `transform`, `transform!` and `combine`).

In [64]:
gdf = groupby(fh5, :symbol)

,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String
1,2020-07-02,257500,17.64,17.74,17.62,17.71,17.71,AAAU
2,2020-07-01,468100,17.73,17.73,17.54,17.68,17.68,AAAU
3,2020-06-30,319100,17.65,17.8,17.61,17.78,17.78,AAAU
4,2020-06-29,405500,17.67,17.69,17.63,17.68,17.68,AAAU
5,2020-06-26,335100,17.49,17.67,17.42,17.67,17.67,AAAU
6,2020-06-25,246800,17.6,17.6,17.52,17.59,17.59,AAAU
7,2020-06-24,329200,17.61,17.71,17.56,17.61,17.61,AAAU
8,2020-06-23,351800,17.55,17.66,17.55,17.66,17.66,AAAU
9,2020-06-22,308300,17.5,17.57,17.44,17.5,17.5,AAAU


In [65]:
gdf_keys = keys(gdf)

6335-element DataFrames.GroupKeys{GroupedDataFrame{DataFrame}}:
 GroupKey: (symbol = "AAAU",)
 GroupKey: (symbol = "AACG",)
 GroupKey: (symbol = "AADR",)
 GroupKey: (symbol = "AAL",)
 GroupKey: (symbol = "AAMC",)
 ⋮
 GroupKey: (symbol = "ZUO",)
 GroupKey: (symbol = "ZVO",)
 GroupKey: (symbol = "ZYME",)
 GroupKey: (symbol = "ZYNE",)
 GroupKey: (symbol = "ZYXI",)

As usual - indexing by a single value drops a dimension (you get a `SubDataFrame`)

In [66]:
gdf[1]

,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String
1,2020-07-02,257500,17.64,17.74,17.62,17.71,17.71,AAAU
2,2020-07-01,468100,17.73,17.73,17.54,17.68,17.68,AAAU
3,2020-06-30,319100,17.65,17.8,17.61,17.78,17.78,AAAU
4,2020-06-29,405500,17.67,17.69,17.63,17.68,17.68,AAAU
5,2020-06-26,335100,17.49,17.67,17.42,17.67,17.67,AAAU
6,2020-06-25,246800,17.6,17.6,17.52,17.59,17.59,AAAU
7,2020-06-24,329200,17.61,17.71,17.56,17.61,17.61,AAAU
8,2020-06-23,351800,17.55,17.66,17.55,17.66,17.66,AAAU
9,2020-06-22,308300,17.5,17.57,17.44,17.5,17.5,AAAU


In [67]:
gdf_keys[1]

GroupKey: (symbol = "AAAU",)

In [68]:
gdf[gdf_keys[1]]

,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String
1,2020-07-02,257500,17.64,17.74,17.62,17.71,17.71,AAAU
2,2020-07-01,468100,17.73,17.73,17.54,17.68,17.68,AAAU
3,2020-06-30,319100,17.65,17.8,17.61,17.78,17.78,AAAU
4,2020-06-29,405500,17.67,17.69,17.63,17.68,17.68,AAAU
5,2020-06-26,335100,17.49,17.67,17.42,17.67,17.67,AAAU
6,2020-06-25,246800,17.6,17.6,17.52,17.59,17.59,AAAU
7,2020-06-24,329200,17.61,17.71,17.56,17.61,17.61,AAAU
8,2020-06-23,351800,17.55,17.66,17.55,17.66,17.66,AAAU
9,2020-06-22,308300,17.5,17.57,17.44,17.5,17.5,AAAU


In [69]:
gdf[(symbol="AAAU",)]

,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String
1,2020-07-02,257500,17.64,17.74,17.62,17.71,17.71,AAAU
2,2020-07-01,468100,17.73,17.73,17.54,17.68,17.68,AAAU
3,2020-06-30,319100,17.65,17.8,17.61,17.78,17.78,AAAU
4,2020-06-29,405500,17.67,17.69,17.63,17.68,17.68,AAAU
5,2020-06-26,335100,17.49,17.67,17.42,17.67,17.67,AAAU
6,2020-06-25,246800,17.6,17.6,17.52,17.59,17.59,AAAU
7,2020-06-24,329200,17.61,17.71,17.56,17.61,17.61,AAAU
8,2020-06-23,351800,17.55,17.66,17.55,17.66,17.66,AAAU
9,2020-06-22,308300,17.5,17.57,17.44,17.5,17.5,AAAU


In [70]:
gdf[("AAAU",)]

,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String
1,2020-07-02,257500,17.64,17.74,17.62,17.71,17.71,AAAU
2,2020-07-01,468100,17.73,17.73,17.54,17.68,17.68,AAAU
3,2020-06-30,319100,17.65,17.8,17.61,17.78,17.78,AAAU
4,2020-06-29,405500,17.67,17.69,17.63,17.68,17.68,AAAU
5,2020-06-26,335100,17.49,17.67,17.42,17.67,17.67,AAAU
6,2020-06-25,246800,17.6,17.6,17.52,17.59,17.59,AAAU
7,2020-06-24,329200,17.61,17.71,17.56,17.61,17.61,AAAU
8,2020-06-23,351800,17.55,17.66,17.55,17.66,17.66,AAAU
9,2020-06-22,308300,17.5,17.57,17.44,17.5,17.5,AAAU


And indexing by a collection produces a subsetted `GroupedDataFrame`:

In [71]:
gdf[1:2]

,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String
1,2020-07-02,257500,17.64,17.74,17.62,17.71,17.71,AAAU
2,2020-07-01,468100,17.73,17.73,17.54,17.68,17.68,AAAU
3,2020-06-30,319100,17.65,17.8,17.61,17.78,17.78,AAAU
4,2020-06-29,405500,17.67,17.69,17.63,17.68,17.68,AAAU
5,2020-06-26,335100,17.49,17.67,17.42,17.67,17.67,AAAU
6,2020-06-25,246800,17.6,17.6,17.52,17.59,17.59,AAAU
7,2020-06-24,329200,17.61,17.71,17.56,17.61,17.61,AAAU
8,2020-06-23,351800,17.55,17.66,17.55,17.66,17.66,AAAU
9,2020-06-22,308300,17.5,17.57,17.44,17.5,17.5,AAAU


In [72]:
gdf[tuple.(["AAAU", "AACG"])]

,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String
1,2020-07-02,257500,17.64,17.74,17.62,17.71,17.71,AAAU
2,2020-07-01,468100,17.73,17.73,17.54,17.68,17.68,AAAU
3,2020-06-30,319100,17.65,17.8,17.61,17.78,17.78,AAAU
4,2020-06-29,405500,17.67,17.69,17.63,17.68,17.68,AAAU
5,2020-06-26,335100,17.49,17.67,17.42,17.67,17.67,AAAU
6,2020-06-25,246800,17.6,17.6,17.52,17.59,17.59,AAAU
7,2020-06-24,329200,17.61,17.71,17.56,17.61,17.61,AAAU
8,2020-06-23,351800,17.55,17.66,17.55,17.66,17.66,AAAU
9,2020-06-22,308300,17.5,17.57,17.44,17.5,17.5,AAAU


Lookup is backed by `Dict` so it is fast.

In [73]:
@btime $gdf[("AACG",)]

  609.756 ns (2 allocations: 2.77 KiB)


,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String
1,2020-07-02,46800,1.3,1.39,1.3,1.31,1.31,AACG
2,2020-07-01,95400,1.25,1.4,1.18,1.31,1.31,AACG
3,2020-06-30,40200,1.16,1.27,1.16,1.26,1.26,AACG
4,2020-06-29,46900,1.15,1.25,1.15,1.17,1.17,AACG
5,2020-06-26,43700,1.12,1.18,1.12,1.15,1.15,AACG
6,2020-06-25,72900,1.22,1.25,1.11,1.23,1.23,AACG
7,2020-06-24,80400,1.14,1.3,1.14,1.25,1.25,AACG
8,2020-06-23,57000,1.19,1.22,1.14,1.17,1.17,AACG
9,2020-06-22,115500,1.0,1.17,0.97,1.14,1.14,AACG


In [74]:
@btime @view $fh5[findall(==("AACG"), $fh5.symbol), :]

  103.738 ms (6852052 allocations: 209.12 MiB)


,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String
1,2020-07-02,46800,1.3,1.39,1.3,1.31,1.31,AACG
2,2020-07-01,95400,1.25,1.4,1.18,1.31,1.31,AACG
3,2020-06-30,40200,1.16,1.27,1.16,1.26,1.26,AACG
4,2020-06-29,46900,1.15,1.25,1.15,1.17,1.17,AACG
5,2020-06-26,43700,1.12,1.18,1.12,1.15,1.15,AACG
6,2020-06-25,72900,1.22,1.25,1.11,1.23,1.23,AACG
7,2020-06-24,80400,1.14,1.3,1.14,1.25,1.25,AACG
8,2020-06-23,57000,1.19,1.22,1.14,1.17,1.17,AACG
9,2020-06-22,115500,1.0,1.17,0.97,1.14,1.14,AACG


In [75]:
@btime filter(:symbol => ==("AACG"), fh5)

  31.082 ms (29 allocations: 865.30 KiB)


,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String
1,2020-07-02,46800,1.3,1.39,1.3,1.31,1.31,AACG
2,2020-07-01,95400,1.25,1.4,1.18,1.31,1.31,AACG
3,2020-06-30,40200,1.16,1.27,1.16,1.26,1.26,AACG
4,2020-06-29,46900,1.15,1.25,1.15,1.17,1.17,AACG
5,2020-06-26,43700,1.12,1.18,1.12,1.15,1.15,AACG
6,2020-06-25,72900,1.22,1.25,1.11,1.23,1.23,AACG
7,2020-06-24,80400,1.14,1.3,1.14,1.25,1.25,AACG
8,2020-06-23,57000,1.19,1.22,1.14,1.17,1.17,AACG
9,2020-06-22,115500,1.0,1.17,0.97,1.14,1.14,AACG


<div class="alert alert-block alert-info">
<b>Tip:</b>
    

Think of `GroupedDataFrame` as a wrapper over a data frame object which caches information
about the parent data frame to make operations that rely on row index faster. Currently this is used in:
* split/apply/combine
* lookup

In other words: if you like row indices in Pandas then `GroupedDataFrame` is a way to achieve such functionality in DataFrames.jl.

Notably:
* you can set multiple row indices to the same data frame, just by creating different `GroupedDataFrame` objects on top of the same data frame.
* the creation of cache in grouped data frame is lazy (it is computed only if needed); cache computation is thread safe.

</div>

In [76]:
for i in 1:3
    local gdf
    @show i
    @time gdf = groupby(fh5, :symbol)
    @time gdf[("AACG",)]
    @time gdf[("AACG",)]
    @time gdf[("AACG",)]
end

i = 1
  0.008684 seconds (66 allocations: 52.287 MiB)
  0.039510 seconds (8.31 k allocations: 52.770 MiB)
  0.000003 seconds (4 allocations: 2.844 KiB)
  0.000002 seconds (4 allocations: 2.844 KiB)
i = 2
  0.007824 seconds (65 allocations: 52.287 MiB)
  0.036424 seconds (8.31 k allocations: 52.770 MiB)
  0.000002 seconds (4 allocations: 2.844 KiB)
  0.000002 seconds (4 allocations: 2.844 KiB)
i = 3
  0.007792 seconds (65 allocations: 52.287 MiB)
  0.036146 seconds (8.31 k allocations: 52.770 MiB)
  0.000002 seconds (4 allocations: 2.844 KiB)
  0.000001 seconds (4 allocations: 2.844 KiB)


## setindex!

`setindex!` is defined only for `DataFrame`, `SubDataFrame` and `DataFrameRow` (other types are read-only).

Intended rules:
* the dimensions of left hand side and right hand side must match;
* if right hand side has names they must match the names in left hand side.

special cases (that might be surprising):

In [77]:
x = rand(5, 5)

5×5 Matrix{Float64}:
 0.372706  0.707413   0.186747   0.331306  0.0668997
 0.631114  0.425126   0.920731   0.793553  0.663167
 0.974898  0.23086    0.0965031  0.766142  0.246591
 0.754282  0.816876   0.22118    0.723069  0.323684
 0.187814  0.0882055  0.730997   0.788639  0.253693

In [78]:
df = DataFrame(x, :auto)

,x1,x2,x3,x4,x5
,Float64,Float64,Float64,Float64,Float64
1,0.372706,0.707413,0.186747,0.331306,0.0668997
2,0.631114,0.425126,0.920731,0.793553,0.663167
3,0.974898,0.23086,0.0965031,0.766142,0.246591
4,0.754282,0.816876,0.22118,0.723069,0.323684
5,0.187814,0.0882055,0.730997,0.788639,0.253693


In [79]:
x[1, 1:4] = [1 2; 3 4]

2×2 Matrix{Int64}:
 1  2
 3  4

In [80]:
x

5×5 Matrix{Float64}:
 1.0       3.0        2.0        4.0       0.0668997
 0.631114  0.425126   0.920731   0.793553  0.663167
 0.974898  0.23086    0.0965031  0.766142  0.246591
 0.754282  0.816876   0.22118    0.723069  0.323684
 0.187814  0.0882055  0.730997   0.788639  0.253693

In [81]:
x[1:4, 1] = [1 2; 3 4]

LoadError: DimensionMismatch("tried to assign 2×2 array to 4×1 destination")

#### Exercise

Check what happens if you try the same operations on `df`.

In [82]:
df

,x1,x2,x3,x4,x5
,Float64,Float64,Float64,Float64,Float64
1,0.372706,0.707413,0.186747,0.331306,0.0668997
2,0.631114,0.425126,0.920731,0.793553,0.663167
3,0.974898,0.23086,0.0965031,0.766142,0.246591
4,0.754282,0.816876,0.22118,0.723069,0.323684
5,0.187814,0.0882055,0.730997,0.788639,0.253693


In [83]:
df[1:4, 1] = [1 2; 3 4]

LoadError: MethodError: no method matching setindex!(::DataFrame, ::Matrix{Int64}, ::UnitRange{Int64}, ::Int64)
[0mClosest candidates are:
[0m  setindex!(::DataFrame, ::Any, [91m::Integer[39m, ::Union{AbstractString, Signed, Symbol, Unsigned}) at /home/bkamins/.julia/packages/DataFrames/nxjiD/src/dataframe/dataframe.jl:647
[0m  setindex!(::DataFrame, [91m::AbstractVector{T} where T[39m, ::AbstractVector{T} where T, ::Union{AbstractString, Signed, Symbol, Unsigned}) at /home/bkamins/.julia/packages/DataFrames/nxjiD/src/dataframe/dataframe.jl:675
[0m  setindex!(::DataFrame, ::AbstractMatrix{T} where T, ::AbstractVector{T} where T, [91m::AbstractVector{T} where T[39m) at /home/bkamins/.julia/packages/DataFrames/nxjiD/src/dataframe/dataframe.jl:727
[0m  ...

In [84]:
df

,x1,x2,x3,x4,x5
,Float64,Float64,Float64,Float64,Float64
1,0.372706,0.707413,0.186747,0.331306,0.0668997
2,0.631114,0.425126,0.920731,0.793553,0.663167
3,0.974898,0.23086,0.0965031,0.766142,0.246591
4,0.754282,0.816876,0.22118,0.723069,0.323684
5,0.187814,0.0882055,0.730997,0.788639,0.253693


In [85]:
df[1, 1:4] = [1 2; 3 4]

2×2 Matrix{Int64}:
 1  2
 3  4

In [86]:
df

,x1,x2,x3,x4,x5
,Float64,Float64,Float64,Float64,Float64
1,1.0,3.0,2.0,4.0,0.0668997
2,0.631114,0.425126,0.920731,0.793553,0.663167
3,0.974898,0.23086,0.0965031,0.766142,0.246591
4,0.754282,0.816876,0.22118,0.723069,0.323684
5,0.187814,0.0882055,0.730997,0.788639,0.253693


<div class="alert alert-block alert-info">
<b>Tip:</b>
    
We want DataFrames.jl to match 100% what Base does with indexing (except for cases where matching column names matters).

If you find cases when it does not please report an issue.

</div>

The most important case, that does work as in Base (as `Matrix` is not resizable) is creation of new columns.

In [87]:
df = DataFrame(rand(2, 2), :auto)

,x1,x2
,Float64,Float64
1,0.917293,0.165872
2,0.652194,0.19013


In [88]:
x3 = [1, 2]

2-element Vector{Int64}:
 1
 2

In [89]:
df.x3_1 = x3

2-element Vector{Int64}:
 1
 2

In [90]:
df[:, :x3_2] = x3

2-element Vector{Int64}:
 1
 2

In [91]:
df[!, :x3_3] = x3

2-element Vector{Int64}:
 1
 2

In [92]:
df

,x1,x2,x3_1,x3_2,x3_3
,Float64,Float64,Int64,Int64,Int64
1,0.917293,0.165872,1,1,1
2,0.652194,0.19013,2,2,2


#### Exercise
Check in which cases `x3` got copied.

In [93]:
eachcol(df[!, r"x3"]) .=== Ref(x3)

3-element BitVector:
 1
 0
 1

A special case when column always gets copied is for range objects:

In [94]:
fh5.col_idx = axes(fh5, 1)

Base.OneTo(6852038)

In [95]:
fh5.col_idx

6852038-element Vector{Int64}:
       1
       2
       3
       4
       5
       ⋮
 6852034
 6852035
 6852036
 6852037
 6852038

Let us try `setindex!` for multi-column indexing.

In [96]:
df

,x1,x2,x3_1,x3_2,x3_3
,Float64,Float64,Int64,Int64,Int64
1,0.917293,0.165872,1,1,1
2,0.652194,0.19013,2,2,2


In [97]:
df2 = DataFrame(x1=[10,20], x2=["a","b"])

,x1,x2
,Int64,String
1,10,a
2,20,b


In [98]:
df[:, 1:2] = df2

LoadError: MethodError: [0mCannot `convert` an object of type [92mString[39m[0m to an object of type [91mFloat64[39m
[0mClosest candidates are:
[0m  convert(::Type{T}, [91m::T[39m) where T<:Number at number.jl:6
[0m  convert(::Type{T}, [91m::Number[39m) where T<:Number at number.jl:7
[0m  convert(::Type{T}, [91m::Base.TwicePrecision[39m) where T<:Number at twiceprecision.jl:250
[0m  ...

Warning: this operation is not atomic now.

In [99]:
df

,x1,x2,x3_1,x3_2,x3_3
,Float64,Float64,Int64,Int64,Int64
1,10.0,0.165872,1,1,1
2,20.0,0.19013,2,2,2


In [100]:
df[!, 1:2] = df2

,x1,x2
,Int64,String
1,10,a
2,20,b


In [101]:
df

,x1,x2,x3_1,x3_2,x3_3
,Int64,String,Int64,Int64,Int64
1,10,a,1,1,1
2,20,b,2,2,2


<div class="alert alert-block alert-info">
<b>Tip:</b>

In `setindex!` context:
* when `:` is used to select rows it operates in-place (it works just as any row selector); except when adding a new column, in which case it copies a column;
* when `!` is used to select rows it always allocates a new column; if a single column is selected - this is a no-copy operation; if multiple columns are selected a copy is always made
* `df.single_col = v` is exactly the same as `df[!, single_col] = v` if `single_col` is a `Symbol` or string literal
* it is not allowed to add columns to `SubDataFrame` or `DataFrameRow`

</div>

## broadcast

Data frame behaves in broadcasting just like a matrix, with two exceptions:
* it forces the style of the result to be a `DataFrame`
* if several data frames take part in broadcasting they must have matching column names

In [102]:
log.(fh5[!, Between(:open, :close)])

,open,high,low,close
,Float64,Float64,Float64,Float64
1,2.87017,2.87582,2.86903,2.87413
2,2.87526,2.87526,2.86448,2.87243
3,2.87074,2.8792,2.86847,2.87807
4,2.87187,2.873,2.8696,2.87243
5,2.86163,2.87187,2.85762,2.87187
6,2.8679,2.8679,2.86334,2.86733
7,2.86847,2.87413,2.86562,2.86847
8,2.86505,2.8713,2.86505,2.8713
9,2.8622,2.86619,2.85877,2.8622


In [103]:
log.(fh5[!, Between(:open, :adjclose)])

LoadError: DomainError with -0.8700000643730164:
log will only return a complex result if called with a complex argument. Try log(Complex(x)).

In [104]:
filter(:adjclose => <=(0), fh5)

,date,volume,open,high,low,close,adjclose,symbol,col_idx
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String,Int64
1,2020-05-13,507400,3.4,3.4,2.99,3.02,-0.87,HCHC,2814814
2,2020-05-12,658600,3.22,3.48,3.11,3.19,-0.918974,HCHC,2814815
3,2020-05-11,564700,3.17,3.43,3.12,3.18,-0.916093,HCHC,2814816
4,2020-05-08,264600,2.92,3.08,2.8,3.06,-0.881523,HCHC,2814817
5,2020-05-07,246900,3.06,3.06,2.67,2.8,-0.806623,HCHC,2814818
6,2020-05-06,219500,2.87,3.08,2.78,2.98,-0.858477,HCHC,2814819
7,2020-05-05,190900,2.7,2.84,2.7,2.8,-0.806623,HCHC,2814820
8,2020-05-04,139800,2.58,2.8,2.58,2.69,-0.774934,HCHC,2814821
9,2020-05-01,288800,2.64,2.77,2.54,2.65,-0.763411,HCHC,2814822


In [105]:
describe(fh5)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,date,,2015-01-02,,2020-07-02,0,Date
2,volume,1.01541e6,1,120600.0,2156725200,0,Int64
3,open,298.086,0.001,24.95,6.91553e7,0,Float64
4,high,305.876,0.0,25.11,7.05886e7,0,Float64
5,low,291.014,0.0,24.75,6.84387e7,0,Float64
6,close,296.783,0.001,24.94,6.95136e7,0,Float64
7,adjclose,293.231,-3.77096,23.3258,6.90222e7,0,Float64
8,symbol,,AAAU,,ZYXI,0,String
9,col_idx,3.42602e6,1,3.42602e6,6852038,0,Int64


#### Exercise

Replace non-positive values in `:adjclose` by a value in `:close` and store it in a variable `:adjclose_fix`.

In [106]:
fh5.adjclose_fix = ifelse.(fh5.adjclose .<= 0, fh5.close, fh5.adjclose)

6852038-element Vector{Float64}:
 17.709999084472656
 17.68000030517578
 17.780000686645508
 17.68000030517578
 17.670000076293945
  ⋮
  0.1363018900156021
  0.1460377424955368
  0.1460377424955368
  0.1557735800743103
  0.1752452850341797

In [107]:
filter(:adjclose => <=(0), fh5)

,date,volume,open,high,low,close,adjclose,symbol,col_idx,adjclose_fix
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String,Int64,Float64
1,2020-05-13,507400,3.4,3.4,2.99,3.02,-0.87,HCHC,2814814,3.02
2,2020-05-12,658600,3.22,3.48,3.11,3.19,-0.918974,HCHC,2814815,3.19
3,2020-05-11,564700,3.17,3.43,3.12,3.18,-0.916093,HCHC,2814816,3.18
4,2020-05-08,264600,2.92,3.08,2.8,3.06,-0.881523,HCHC,2814817,3.06
5,2020-05-07,246900,3.06,3.06,2.67,2.8,-0.806623,HCHC,2814818,2.8
6,2020-05-06,219500,2.87,3.08,2.78,2.98,-0.858477,HCHC,2814819,2.98
7,2020-05-05,190900,2.7,2.84,2.7,2.8,-0.806623,HCHC,2814820,2.8
8,2020-05-04,139800,2.58,2.8,2.58,2.69,-0.774934,HCHC,2814821,2.69
9,2020-05-01,288800,2.64,2.77,2.54,2.65,-0.763411,HCHC,2814822,2.65


and something more advanced now for each stock calculate a column with log-returns of `:close` and store it in `:close_returns`:

In [108]:
df = sort(fh5, :date)

,date,volume,open,high,low,close,adjclose,symbol,col_idx,adjclose_fix
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String,Int64,Float64
1,2015-01-02,2000,37.25,37.25,36.64,36.64,35.3998,AADR,2114,35.3998
2,2015-01-02,10748600,54.28,54.6,53.07,53.91,51.0799,AAL,3499,51.0799
3,2015-01-02,11500,308.0,348.59,308.0,327.18,327.18,AAMC,4798,327.18
4,2015-01-02,11400,3.99,4.03,3.98,4.03,3.91772,AAME,6079,3.91772
5,2015-01-02,898900,30.81,30.86,30.04,30.62,30.0588,AAN,7464,30.0588
6,2015-01-02,184600,11.28,11.28,10.72,10.79,10.79,AAOI,8840,10.79
7,2015-01-02,90700,22.55,22.68,21.6,21.93,20.9958,AAON,10225,20.9958
8,2015-01-02,509800,160.85,162.5,157.47,158.56,156.251,AAP,11610,156.251
9,2015-01-02,53204600,111.39,111.44,107.35,109.33,99.9459,AAPL,12995,99.9459


In [109]:
gdf = groupby(df, :symbol)

,date,volume,open,high,low,close,adjclose,symbol,col_idx,adjclose_fix
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String,Int64,Float64
1,2018-08-15,27300,11.84,11.84,11.74,11.74,11.74,AAAU,474,11.74
2,2018-08-16,428400,11.78,11.8,11.74,11.74,11.74,AAAU,473,11.74
3,2018-08-17,52400,11.8,11.82,11.77,11.82,11.82,AAAU,472,11.82
4,2018-08-20,28700,11.88,11.91,11.85,11.9,11.9,AAAU,471,11.9
5,2018-08-21,30600,11.92,11.95,11.89,11.93,11.93,AAAU,470,11.93
6,2018-08-22,101200,11.98,11.98,11.94,11.96,11.96,AAAU,469,11.96
7,2018-08-23,54800,11.91,11.92,11.85,11.85,11.85,AAAU,468,11.85
8,2018-08-24,106500,11.97,12.08,11.96,12.05,12.05,AAAU,467,12.05
9,2018-08-27,295100,12.06,12.13,12.06,12.1,12.1,AAAU,466,12.1


In [110]:
df2 = transform(gdf, :close => (x -> log.(x ./ lag(x))) => :close_returns)

,date,volume,open,high,low,close,adjclose,symbol,col_idx,adjclose_fix,close_returns
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String,Int64,Float64,Float64?
1,2015-01-02,2000,37.25,37.25,36.64,36.64,35.3998,AADR,2114,35.3998,missing
2,2015-01-02,10748600,54.28,54.6,53.07,53.91,51.0799,AAL,3499,51.0799,missing
3,2015-01-02,11500,308.0,348.59,308.0,327.18,327.18,AAMC,4798,327.18,missing
4,2015-01-02,11400,3.99,4.03,3.98,4.03,3.91772,AAME,6079,3.91772,missing
5,2015-01-02,898900,30.81,30.86,30.04,30.62,30.0588,AAN,7464,30.0588,missing
6,2015-01-02,184600,11.28,11.28,10.72,10.79,10.79,AAOI,8840,10.79,missing
7,2015-01-02,90700,22.55,22.68,21.6,21.93,20.9958,AAON,10225,20.9958,missing
8,2015-01-02,509800,160.85,162.5,157.47,158.56,156.251,AAP,11610,156.251,missing
9,2015-01-02,53204600,111.39,111.44,107.35,109.33,99.9459,AAPL,12995,99.9459,missing


In [111]:
combine(sdf -> first(sdf, 3), groupby(df2, :symbol), ungroup=false)

,symbol,date,volume,open,high,low,close,adjclose,col_idx,adjclose_fix,close_returns
,String,Date,Int64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Float64?
1,AAAU,2018-08-15,27300,11.84,11.84,11.74,11.74,11.74,474,11.74,missing
2,AAAU,2018-08-16,428400,11.78,11.8,11.74,11.74,11.74,473,11.74,0.0
3,AAAU,2018-08-17,52400,11.8,11.82,11.77,11.82,11.82,472,11.82,0.00679119
,symbol,date,volume,open,high,low,close,adjclose,col_idx,adjclose_fix,close_returns
,String,Date,Int64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Float64?
1,ZYXI,2015-01-02,100,0.18,0.18,0.18,0.18,0.175245,6852038,0.175245,missing
2,ZYXI,2015-01-05,33200,0.22,0.22,0.14,0.16,0.155774,6852037,0.155774,-0.117783
3,ZYXI,2015-01-07,8100,0.14,0.15,0.13,0.15,0.146038,6852036,0.146038,-0.0645385


#### Exercise

Check our codes to make sure that in each group in `df2` first and only the fist element of `:close_returns` is `missing`

In [112]:
all(sdf -> findall(ismissing, sdf.close_returns) == [1], groupby(df2, :symbol))

true

#### Exercises

Check what happens if you try to broadcast a sum of a 1-row `DataFrame` with an array having multiple rows

In [113]:
[1 2] .+ rand(2,2)

2×2 Matrix{Float64}:
 1.60789  2.67301
 1.5804   2.88401

In [114]:
DataFrame([1 2], :auto) .+ rand(2,2)

,x1,x2
,Float64,Float64
1,1.39059,2.26281
2,1.26122,2.99972


Check what happens if you try to broadcast a sum of a 0-row `DataFrame` with an array having multiple rows

In [115]:
zeros(0, 2) .+ rand(2,2)

LoadError: DimensionMismatch("arrays could not be broadcast to a common size; got a dimension with lengths 0 and 2")

In [116]:
DataFrame(x1=[], x2=[]) .+ rand(2,2)

LoadError: DimensionMismatch("arrays could not be broadcast to a common size; got a dimension with lengths 0 and 2")

Check what happens when you try to broadcast a sum of a `DataFrame` with a 3D array.

In [117]:
[1 2] .+ rand(2,2,2)

2×2×2 Array{Float64, 3}:
[:, :, 1] =
 1.13849  2.42206
 1.76999  2.85878

[:, :, 2] =
 1.81264  2.06532
 1.03624  2.81627

In [118]:
DataFrame([1 2], :auto) .+ rand(2,2,2)

LoadError: DimensionMismatch("cannot broadcast a data frame into 3 dimensions")

Check if broadcasting is defined for `DataFrameRow`.

In [119]:
dfr = DataFrame(x1=1, x2=2)[1, :]

,x1,x2
,Int64,Int64
1,1,2


In [120]:
dfr .+ 1

LoadError: ArgumentError: broadcasting over `DataFrameRow`s is reserved

In [121]:
nt = NamedTuple(dfr)

(x1 = 1, x2 = 2)

In [122]:
nt .+ 1

LoadError: ArgumentError: broadcasting over dictionaries and `NamedTuple`s is reserved

<div class="alert alert-block alert-info">
<b>Tip:</b>

In broadcasting `df.single_col` and `df[!, single_col]` behave in the same way (the same exceptions as in `getindex` apply).

</div>

## broadcast!

It is possible to assign a value to `AbstractDataFrame` and `DataFrameRow` objects using the `.=` operator.

In such an operation `AbstractDataFrame` is considered as two-dimensional and `DataFrameRow` as single-dimensional (columnar).

#### Special cases:

Broadcasting into a single cell unwraps it before opertion:

In [123]:
df = DataFrame(a=[[1,2], [3,4]])

,a
,Array…
1,"[1, 2]"
2,"[3, 4]"


In [124]:
df[1,1] .= [10, 20]

2-element Vector{Int64}:
 10
 20

In [125]:
df

,a
,Array…
1,"[10, 20]"
2,"[3, 4]"


Broadcasting into any slice of a data frame is in-place except for two cases:
* `!` is used as a row selector
* `:` is used as a row selector and column selector is a single column that does not exist in a data frame

In these cases a new column is allocated, except if a data frame is `SubDataFrame` in which case column adding is disallowed.

<div class="alert alert-block alert-info">
<b>Tip:</b>

These rules mean that `df.single_col .= v` behaves in the same way as `df[:, single_col] .= v`

(NOTE that here we use `:` not `!` in this context),

except that `df.single_col .= v` does not allow creation of new columns, integer indexing, nor using a variable to pass a name.

</div>

## Summary

All the rules how indexing works in DataFrames.jl are specified here:

https://juliadata.github.io/DataFrames.jl/latest/lib/indexing/

If you find some cases where the behavior does not match these rules please report an issue.

<div class="alert alert-block alert-info">
<b>Tip:</b>

Rules for most common operations:
* get a column without copying `df.single_col` or `df[!, single_col]`
* get a column with copying `df[:, single_col]`


* assign a column without copying `df.single_col = vector` or `df[!, single_col] = vector` (except for ranges)
* assign a column with copying `df[:, single_col] = vector`
* assign a value in-place with broadcasting if column exists:  `df.single_col .= value` or `df[:, single_col] .= value`
* assign a value with copying, create a column if it does not exist: `df[!, single_col] .= value`
* for convenience also `df[:, single_col] .= value` creates a column with copying if it does not exist

</div>